In [ ]:
import datetime
from dateutil.relativedelta import relativedelta
import requests
import pandas as pd


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Se si usa solo un sottoinsieme (chosen airports) conviene usare questo codice così non fa doppioni su dataset, perche codice fa andata ritorno
chosen_airports = ['BVA', 'BCN']

import itertools
routes = []
for subset in itertools.combinations(chosen_airports, 2):
    routes.append(subset)
print(routes)

[('STN', 'BCN')]


In [ ]:
# Get flights information for the airports chosen

today = datetime.datetime.today()
master_df = pd.DataFrame(columns = ['Origin','Destination','Period','Date','Time','Price','Published Price'])
date_dict = {}

for keys, values in routes:
    for i in range(34):
        day = today + relativedelta(days=+i)
        date_dict[str(i) + 'day'] = day
    two_months = today + relativedelta(months=+2)
    date_dict['Two Months'] = two_months
    three_months = today + relativedelta(months=+3)
    date_dict['Three Months'] = three_months
    four_month = today + relativedelta(months=+4)
    date_dict['Four Months'] = four_month
    for keys2, values2  in date_dict.items():
        date_in = values2.strftime('%Y-%m-%d')
        date_out = values2 + relativedelta(days=+0)
        date_out = date_out.strftime('%Y-%m-%d')
        url = 'https://www.ryanair.com/api/booking/v4/en-gb/availability?ADT=1&CHD=0&DateIn={}&DateOut={}&Destination={}&Disc=0&INF=0&Origin={}&TEEN=0&promoCode=&IncludeConnectingFlights=false&RoundTrip=true&ToUs=AGREED'.format(date_in, date_out, values, keys)
        content = str(requests.get(url).content)
        flight_keys, prices, published_prices = ([] for i in range(3)) 
        for i, _ in enumerate(content):
            needle1 = '"flightKey":"'
            needle2 = '"amount":'
            needle3 = '"publishedFare":'
            if content[i:i + len(needle1)] == needle1:
                flight_keys.append(content[i+13:i+200].split('~~","',1)[0])
            if content[i:i + len(needle2)] == needle2:
                prices.append(content[i+9:i+18].split(',',1)[0])
            if content[i:i + len(needle3)] == needle3:
                published_prices.append(content[i+16:i+25].split(',',1)[0])
        if len(flight_keys) == len(published_prices) == len(prices):
            for i in range(0, len(flight_keys)):
                time = flight_keys[i][-5:]
                date  = flight_keys[i][-16:-6]
                if flight_keys[i][-20:-17] == values:
                    origin = keys
                    destination = values
                else:
                    origin = values
                    destination = keys
                price = prices[i]
                published_price = published_prices[i]
                row = [origin,destination,keys2,date,time,price,published_price]
                master_df.loc[len(master_df)] = row
print(master_df)

    Origin Destination        Period        Date   Time   Price  \
0      STN         BCN          0day  04/21/2022  23:45   97.49   
1      STN         BCN          1day  04/22/2022  11:25  126.99   
2      STN         BCN          1day  04/22/2022  15:55  120.49   
3      STN         BCN          1day  04/22/2022  19:10  138.99   
4      STN         BCN          1day  04/22/2022  23:45  102.99   
..     ...         ...           ...         ...    ...     ...   
410    BLQ         STN  Three Months  07/21/2022  21:35   16.99   
411    STN         BLQ   Four Months  08/21/2022  10:35   29.79   
412    STN         BLQ   Four Months  08/21/2022  23:35   49.49   
413    BLQ         STN   Four Months  08/21/2022  07:10   29.79   
414    BLQ         STN   Four Months  08/21/2022  20:10   32.99   

    Published Price  
0            108.99  
1            126.99  
2            120.49  
3            138.99  
4            102.99  
..              ...  
410           16.99  
411           32.99

In [ ]:
now = datetime.datetime.now() # current date and time

year = now.strftime("%Y")
month = now.strftime("%m")
day = now.strftime("%d")
time = now.strftime("%H:%M:%S")
date_time = now.strftime("%m%d%Y-%H:%M:%S")

In [ ]:
master_df.to_csv('/content/drive/MyDrive/data/BVABCN/'+date_time+'.csv')